In [1]:
import re
from functools import lru_cache
import time
import pandas as pd
from pandas import DataFrame
import pandasql
import rolling_pin.blob_etl as rpb
import cufflinks as cf
import pyparsing as pp
cf.go_offline()
cf.set_config_file(theme='henanigans', colorscale='henanigans')

from shekels.core.database import Database
import shekels.core.data_tools as sdt

In [2]:
mich = '/home/ubuntu/shekels/resources/mich-transactions.csv'
alex = '/home/ubuntu/shekels/resources/alex-transactions.csv'
mich = pd.read_csv(mich)
mich['Notes'] = 'mich'
alex = pd.read_csv(alex)
alex['Notes'] = 'alex'
combo = pd.concat([alex, mich], axis=0, ignore_index=True)
target = '/home/ubuntu/shekels/resources/combo-transactions.csv'
combo.to_csv(target, index=False)

In [ ]:
config = '/home/ubuntu/shekels/resources/config.json'
db = Database.from_json(config).update()
data = db.data

In [13]:
def get_mintapi_data(source):
    with open(source) as f:
        data = json.load(f)

    lut = dict(
        odate='date',
        merchant='description',
        omerchant='original_description',
        amount='amount',
        account='account',
        category='category',
        labels='labels',
        note='notes',
        fi='financial_institution'
    )
    data = DataFrame(data)
    del data['date']
    data.rename(
        lambda x: lut[x] if x in lut.keys() else x,
        axis=1,
        inplace=True
    )
    data.rename(lbt.as_snakecase, axis=1, inplace=True)
    data.date = data.date.apply(
        lambda x: datetime.utcfromtimestamp(int(str(x)[:-3]))
    )
#     data = data[lut.values()]
    return data


#     lut = get_periodicity_table(data, 'date', 'description', 'amount')
def get_periodicity_table(data, group_column, key_column, value_column):
    data = data.copy()
    data[group_column] = data[group_column].apply(
        lambda x: x.replace(microsecond=randint(0, 1000000))
    )
    cols = [group_column, key_column, value_column]
    data = data[cols].pivot(*cols)
    data[group_column] = data.index
    data.reset_index(drop=True, inplace=True)

    score = DataFrame()
    date_intervals = [
        'year',
        'half_year',
        'quarter_year',
        'month',
        'half_month',
        'week',
        'day',
        'half_day',
    ]
    for period in date_intervals:
        prob = data.copy()
        prob[group_column] = prob[group_column].apply(lambda x: conform_date(x, period))
        prob = prob.groupby(group_column).count()
        cnt = prob.sum()
        prob = prob.applymap(lambda x: 0 if x == 0 else 1.0 / x)
        prob = prob.mean() * cnt
        score[period] = prob

    score = score.T
    output = DataFrame()
    output[key_column] = score.columns.tolist()
    output['period'] = score.apply(lambda x: np.argmax(x)).tolist()
    output['score'] = score.apply(lambda x: np.max(x)).tolist()
    output.sort_values('score', ascending=False, inplace=True)
    output.reset_index(drop=True, inplace=True)
    return output


def date_pivot(data, key_column, value_column):
    data.date = data.date.apply(lambda x: x.replace(microsecond=randint(0, 1000000)))
    cols = ['date', key_column, value_column]
    data = data[cols].pivot(*cols)
    data['date'] = data.index.tolist()
    data.date = data.date.apply(lambda x: x.replace(microsecond=0))
    data.reset_index(drop=True, inplace=True)
    return data


def conform_date(date, period):
    if period == 'year':
        return datetime(date.year, 1, 1)

    if period == 'half_year':
        if date.month >= 7:
            return datetime(date.year, 7, 1)
        return datetime(date.year, 1, 1)

    if period == 'quarter_year':
        if date.month >= 10:
            return datetime(date.year, 10, 1)
        elif date.month >= 7:
            return datetime(date.year, 7, 1)
        elif date.month < 4:
            return datetime(date.year, 4, 1)
        else:
            return datetime(date.year, 1, 1)

    if period == 'month':
        return datetime(date.year, date.month, 1)

    if period == 'half_month':
        if date.day >= 16:
            return datetime(date.year, date.month, 16)
        return datetime(date.year, date.month, 1)

    if period == 'week':
        if date.day >= 22:
            return datetime(date.year, date.month, 22)
        elif date.day >= 15:
            return datetime(date.year, date.month, 15)
        elif date.day >= 8:
            return datetime(date.year, date.month, 8)
        else:
            return datetime(date.year, date.month, 1)

    if period == 'day':
        return datetime(date.year, date.month, date.day)

    if period == 'half_day':
        if date.hour >= 13:
            return datetime(date.year, date.month, date.day, 13)
        return datetime(date.year, date.month, date.day, 1)

    raise ValueError(period + ' is not a valid period')
